## Text analysis

1. Determine bias of the input article
2. Determine direction (left/right)





BIAS DETECTION:
    - detect biased words, compute an index
    - Get sentiment on various topics, known to differ from left to right political view (sentiment analysis on paragraphs)

In [51]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [20]:
# Read in the sample article
article = open('sample_article.txt', 'r').read()

# Convert to single long string
article = article.replace('\'', ' ')

In [21]:
article

'Virginia mom says activists threatened to  curb stomp  her for speaking out against school curriculum\n This is murder,  Loudoun County parent Alisha Brand told  Fox & Friends \n\nParents in Loudoun County, Virginia, say they have become targets of violent threats on social media for speaking out against the school district s LGBTQ agenda.\n\n"They said that they wanted to curb stomp me," Loudoun County mom and Army of Parents co-founder Alisha Brand told "Fox & Friends" Friday of threats made on the "Loudoun Love Warriors" Facebook page.\n\nBrand said her organization, which she described as a 501(c)(4) that advocates for excellence in education, school safety and parental rights, has made activists online "very angry," leading to "threats of death."\n\n"I m not sure if your audience is aware of the violent nature of curb stomping, but what it does entail is grabbing somebody by the back of the head, forcing their mouth open, pushing them down to the ground with their face on the cem

### Preprocess text

In [32]:
# Download required NLTK data
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [52]:
def preprocess_text(text):

    # Remove stopwords
    tokens = word_tokenize(text)
    tokens = re.sub(r'[^\w\s]', '', " ".join(tokens))
    tokens = tokens.split(" ")
    stop_words = stopwords.words('english')
    tokens = [token.lower() for token in tokens if token.lower() not in stop_words and len(token) > 0]

    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

In [53]:
lemmatized_tokens = preprocess_text(article)

#### Test sentiment analysis with chatGPT API

In [64]:
import openai

In [74]:
def get_sentiment(topic: str, text: str) -> str:
    
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=f"Decide whether text's sentiment is positive, neutral, or negative based on {topic}.\n\n text: {text} \"\nSentiment:",
    temperature=0,
    max_tokens=100,
    top_p=1.0,
    frequency_penalty=0.5,
    presence_penalty=0.0
    )

In [72]:
openapi_key = open("openai_api_key", "r").read()
import os

openai.api_key = openapi_key.strip()

import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")



In [73]:
response

<OpenAIObject text_completion id=cmpl-7FjOEEP8JT4QgdchYZlEze6DE2Qdy at 0x1d6fa5790d0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " Positive"
    }
  ],
  "created": 1683983754,
  "id": "cmpl-7FjOEEP8JT4QgdchYZlEze6DE2Qdy",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 31,
    "total_tokens": 32
  }
}